In [4]:
import time as time
import datetime as datetime
import pandas as pd
import yfinance as yf
import matplotlib as plt
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [31]:
#IMPORT LIST OF SP500 STOCKS HERE IN THE TICKER VARIABLE TO RUN TESTS ON ALL OF THE INDIVIDUAL 
#STOCKS

#Ticker in url
ticker = 'TSLA'

#Timeperiods of data set "Y/M/D/time"
#this will help when selecting the desired dates and will pull the data set from yahoo finance.
period1 = int(time.mktime(datetime.datetime(2020, 1, 2, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2023, 1, 6, 23, 59).timetuple()))
interval = '1d' # 1wk, 1m

#Yahoo Finance url
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'

#defining df to backtesting requirements
df = pd.read_csv(url)
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'adj close', 'Volume']
df.columns = columns

#index df to date time index to fit backtesting.py
#df requirements
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

#dropping columns that are not necessary for backtesting.py format
df.drop('Date', inplace=True, axis=1)
df.drop('adj close', inplace=True, axis=1)
df

,Open,High,Low,Close,Volume
2020-01-02,28.299999,28.713333,28.114000,28.684000,142981500
2020-01-03,29.366667,30.266666,29.128000,29.534000,266677500
2020-01-06,29.364668,30.104000,29.333332,30.102667,151995000
2020-01-07,30.760000,31.441999,30.224001,31.270666,268231500
2020-01-08,31.580000,33.232666,31.215334,32.809334,467164500
...,...,...,...,...,...
2022-12-30,119.949997,124.480003,119.750000,123.180000,157304500
2023-01-03,118.470001,118.800003,104.639999,108.099998,231402800
2023-01-04,109.110001,114.589996,107.519997,113.639999,180389000
2023-01-05,110.510002,111.750000,107.160004,110.339996,157986300


In [32]:
#the overall trading strategy function
class MACD(Strategy): 
     
    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):
      
        #MACD variables
        self.macd, self.macdsignal, self.macdhist = self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        #A comparison from the TA-lib directory 
        #macd, macdsignal, macdhist = MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
       
    def next(self):
        
        if crossover(self.macdsignal, self.macd):
            #if this statement is true the below command signals a sell.
            self.position.close()
            self.sell()
        
        elif crossover(self.macd, self.macdsignal):
            #buy command 
            self.position.close()
            self.buy()
                       

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to 
#backtesting.py on github
bt = Backtest(df, MACD, cash = 10_000)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be 
#run in a earlier python like python 3.6 to graph the trades 
#bt.plot()

Start                     2020-01-02 00:00:00
End                       2023-01-06 00:00:00
Duration                   1100 days 00:00:00
Exposure Time [%]                   95.131579
Equity Final [$]                  8540.759063
Equity Peak [$]                  21607.735074
Return [%]                         -14.592409
Buy & Hold Return [%]              294.157014
Return (Ann.) [%]                   -5.095748
Volatility (Ann.) [%]               65.550147
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -78.97645
Avg. Drawdown [%]                  -19.830276
Max. Drawdown Duration      973 days 00:00:00
Avg. Drawdown Duration      148 days 00:00:00
# Trades                                   59
Win Rate [%]                        33.898305
Best Trade [%]                      52.863243
Worst Trade [%]                    -32.333803
Avg. Trade [%]                    

In [34]:
#the overall trading strategy function
class STOC(Strategy): 
     
    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):
      
       
        #STOCH variables
        self.slowk, self.slowd = self.I(talib.STOCH, self.data.High, self.data.Low, self.data.Close, fastk_period=5, slowk_period=3, slowd_period=3)
    
        #slowk, slowd = STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    def next(self):
        
        if crossover(self.slowk, self.slowd):
            #if this statement is true the below command signals a sell.
            self.position.close()
            self.sell()
        
        elif crossover(self.slowd, self.slowk):
            #buy command 
            self.position.close()
            self.buy()
                       

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to 
#backtesting.py on github
bt = Backtest(df, STOC, cash = 10_000)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be 
#run in a earlier python like python 3.6 to graph the trades 
#bt.plot()

Start                     2020-01-02 00:00:00
End                       2023-01-06 00:00:00
Duration                   1100 days 00:00:00
Exposure Time [%]                   98.552632
Equity Final [$]                  3682.099707
Equity Peak [$]                   13646.14389
Return [%]                         -63.179003
Buy & Hold Return [%]              294.157014
Return (Ann.) [%]                  -28.199674
Volatility (Ann.) [%]               62.434818
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -78.475253
Avg. Drawdown [%]                  -39.467156
Max. Drawdown Duration     1072 days 00:00:00
Avg. Drawdown Duration      537 days 00:00:00
# Trades                                  195
Win Rate [%]                        56.923077
Best Trade [%]                      26.048742
Worst Trade [%]                    -38.796827
Avg. Trade [%]                    

In [36]:
#the overall trading strategy function
class MACDSTOC(Strategy): 
    stoc_buy = False
    stoc_sell = False
    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):
      
       
        #STOCH variables
        self.slowk, self.slowd = self.I(talib.STOCH, self.data.High, self.data.Low, self.data.Close, fastk_period=5, slowk_period=3, slowd_period=3)
        self.macd, self.macdsignal, self.macdhist = self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
       
        #slowk, slowd = STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    def next(self):
        
        if crossover(self.slowd, self.slowk):
            self.stoc_buy = True
            self.stoc_sell = False
        elif crossover(self.slowk, self.slowd):
            self.stoc_sell = True
            self.stoc_buy = False
            
        if self.stoc_sell and crossover(self.macdsignal, self.macd):
            #if this statement is true the below command signals a sell.
            self.position.close()
            self.sell()
        elif self.stoc_buy and crossover(self.macd, self.macdsignal):
            #buy command 
            self.position.close()
            self.buy()
            

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to 
#backtesting.py on github
bt = Backtest(df, MACDSTOC, cash = 10_000)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be 
#run in a earlier python like python 3.6 to graph the trades 
#bt.plot()

Start                     2020-01-02 00:00:00
End                       2023-01-06 00:00:00
Duration                   1100 days 00:00:00
Exposure Time [%]                   92.105263
Equity Final [$]                 42619.897391
Equity Peak [$]                  63504.533397
Return [%]                         326.198974
Buy & Hold Return [%]              294.157014
Return (Ann.) [%]                   61.720925
Volatility (Ann.) [%]               130.63972
Sharpe Ratio                         0.472451
Sortino Ratio                        1.314198
Calmar Ratio                         0.844101
Max. Drawdown [%]                  -73.120306
Avg. Drawdown [%]                   -9.456443
Max. Drawdown Duration      759 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                     515.558724
Worst Trade [%]                    -36.970743
Avg. Trade [%]                    

In [37]:
bt.plot()

Row(id='8878', ...)

In [38]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,293,60,243,34.017334,209.396667,51386.144569,5.155587,2020-03-30,2020-12-17,262 days
1,-293,243,267,209.396667,285.000000,-22151.776569,-0.361053,2020-12-17,2021-01-25,39 days
2,-137,267,383,285.000000,220.733337,8804.532831,0.225497,2021-01-25,2021-07-12,168 days
3,-217,383,386,220.733337,219.463333,275.590868,0.005754,2021-07-12,2021-07-15,3 days
4,-220,386,445,219.463333,261.820007,-9318.468280,-0.193001,2021-07-15,2021-10-07,84 days
5,-148,445,447,261.820007,262.549988,-108.037188,-0.002788,2021-10-07,2021-10-11,4 days
6,-148,447,512,262.549988,359.616669,-14365.868788,-0.369707,2021-10-11,2022-01-12,93 days
7,-68,512,514,359.616669,339.959991,1336.654104,0.054660,2022-01-12,2022-01-14,2 days
8,-76,514,531,339.959991,311.666656,2150.293460,0.083225,2022-01-14,2022-02-09,26 days
9,89,531,658,311.666656,289.416656,-1980.250000,-0.071390,2022-02-09,2022-08-12,184 days
